# Meeting 03.05.2024 Christoph-Nils-Arne
Varianz vorhersagen ist der Kern

einfaches Regressionsmodell 

GARCH
Nils LSTM

Kaggle aufsetzen 
Copilot

### Packages:
- AEON 
- SK Time 
- DARts 

optimal stopping 

LIteratur:
- https://rdcu.be/dGI38

In [1]:
# pip install openpyxl

In [2]:
# pip install arch

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import numpy as np
from matplotlib import pyplot
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf

# Nils - Arne 16.05.24
- extract USD - EURO pair from the dataset
- LinearRegression (ARne)
- GARCH(Nils)



In [4]:

file_path = 'data\BB_Rates_15min_from_10-2023.xlsx'
data = pd.read_excel(file_path)



<>:1: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\arnet\AppData\Local\Temp\ipykernel_13340\4088575790.py:1: SyntaxWarning: invalid escape sequence '\B'
  file_path = 'data\BB_Rates_15min_from_10-2023.xlsx'


In [5]:
start_index = 3
EURUSD = data.iloc[start_index:, :2]

# Rename the columns
EURUSD.columns = ['Dates', 'Close']

# Display the first few rows of the new dataset to verify
#print(EURUSD.head())

                 Dates   Close
3  2023-10-03 23:00:00  1.0469
4  2023-10-03 23:15:00  1.0466
5  2023-10-03 23:30:00  1.0465
6  2023-10-03 23:45:00  1.0465
7  2023-10-04 00:00:00  1.0468


In [6]:
# Convert the 'Dates' column to datetime format
EURUSD['Dates'] = pd.to_datetime(EURUSD['Dates'])

# Calculate the difference between consecutive dates
EURUSD['TimeDiff'] = EURUSD['Dates'].diff()

# Define the 15-minute time interval
time_interval = pd.Timedelta(minutes=15)

# Identify the sections with consecutive 15-minute intervals
EURUSD['IsConsecutive'] = EURUSD['TimeDiff'] == time_interval

# Add a column to identify the observation blocks
EURUSD['Block'] = (EURUSD['IsConsecutive'] != EURUSD['IsConsecutive'].shift()).cumsum()

# Filter out sections with fewer than 2 days of consecutive 15-minute observations
min_consecutive_minutes = 2 * 24 * 60  # 2 days in minutes
valid_blocks = EURUSD.groupby('Block').filter(lambda x: len(x) >= min_consecutive_minutes / 15).Block.unique()
EURUSD_filtered = EURUSD[EURUSD['Block'].isin(valid_blocks)]

# Add the observation timeframe column
EURUSD_filtered['TimeFrame'] = EURUSD_filtered.groupby('Block').cumcount() + 1

# Drop the 'TimeDiff' and 'IsConsecutive' columns as they are no longer needed
EURUSD_filtered = EURUSD_filtered.drop(columns=['TimeDiff', 'IsConsecutive'])

# Display the first few rows of the filtered dataset with the new 'Block' and 'TimeFrame' columns
#print(EURUSD_filtered.head())

                Dates   Close  Block  TimeFrame
4 2023-10-03 23:15:00  1.0466      2          1
5 2023-10-03 23:30:00  1.0465      2          2
6 2023-10-03 23:45:00  1.0465      2          3
7 2023-10-04 00:00:00  1.0468      2          4
8 2023-10-04 00:15:00  1.0467      2          5


C:\Users\arnet\AppData\Local\Temp\ipykernel_13340\3678701610.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EURUSD_filtered['TimeFrame'] = EURUSD_filtered.groupby('Block').cumcount() + 1


In [7]:
# Create a mapping from original block numbers to sequential block numbers
block_mapping = {old_block: new_block for new_block, old_block in enumerate(EURUSD_filtered['Block'].unique(), 1)}


In [8]:
# Apply the mapping to the Block column
EURUSD_filtered['Block'] = EURUSD_filtered['Block'].map(block_mapping)


In [9]:
EURUSD_filtered = EURUSD_filtered.reset_index(drop=True)
EURUSD_filtered = EURUSD_filtered.drop(columns=['TimeFrame'])



In [11]:
#print(EURUSD_filtered.head())
#print(EURUSD_filtered.tail())

                Dates   Close  Block
0 2023-10-03 23:15:00  1.0466      1
1 2023-10-03 23:30:00  1.0465      1
2 2023-10-03 23:45:00  1.0465      1
3 2023-10-04 00:00:00  1.0468      1
4 2023-10-04 00:15:00  1.0467      1
                    Dates   Close  Block
13428 2024-04-17 09:30:00  1.0632     29
13429 2024-04-17 09:45:00  1.0625     29
13430 2024-04-17 10:00:00   1.063     29
13431 2024-04-17 10:15:00  1.0642     29
13432 2024-04-17 10:30:00  1.0647     29


In [12]:
print("preperation complete!")

preperation complete
